# SageMaker JumpStart - 이미지 생성

***
Amazon [SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html)에 오신 것을 환영합니다! JumpStart를 사용하면 SageMaker Studio에서 원클릭으로 또는 [SageMaker JumpStart API](https://sagemaker.readthedocs.io/en/stable/overview.html#use-prebuilt-models-with-sagemaker-jumpstart)를 통해 다양한 머신 러닝 작업을 해결할 수 있습니다. 이 데모 노트북에서는 JumpStart API를 사용하여 최첨단 Stable Diffusion 모델로 텍스트에서 이미지를 생성하는 방법을 보여드립니다. 또한 모델을 사용자의 데이터셋에 맞게 미세 조정하는 방법도 보여드립니다.

Stable Diffusion은 텍스트 프롬프트만으로 사실적인 이미지를 생성할 수 있게 해주는 텍스트-이미지 변환 모델입니다. 확산 모델은 실제 이미지에 추가된 노이즈를 제거하는 방법을 학습함으로써 훈련됩니다. 이 노이즈 제거 과정은 사실적인 이미지를 생성합니다. 이러한 모델은 생성 과정을 텍스트에 조건화함으로써 텍스트만으로도 이미지를 생성할 수 있습니다. 예를 들어, Stable Diffusion은 잠재 확산 모델로, 순수한 노이즈 이미지에서 형태를 인식하고 이러한 형태가 입력 텍스트의 단어와 일치하면 점진적으로 이 형태들을 선명하게 만듭니다.

Stable Diffusion과 같은 대규모 모델을 훈련하고 배포하며 추론을 실행하는 것은 종종 cuda 메모리 부족, 페이로드 크기 제한 초과 등과 같은 문제를 포함하여 어려운 과정입니다. JumpStart는 철저히 테스트된 즉시 사용 가능한 스크립트를 제공함으로써 이 과정을 단순화합니다. 또한 권장 인스턴스 유형, 이미지 생성 과정을 안내하는 매개변수 선택 방법, 프롬프트 엔지니어링 등 프로세스의 각 단계에 대한 지침을 제공합니다. 더욱이, 자체 코드를 작성하지 않고도 JumpStart의 80개 이상의 확산 모델 중 어느 것이든 배포하고 추론을 실행할 수 있습니다.

이 노트북에서는 JumpStart를 사용하여 어떤 주제/물체/환경/장면이든 매우 사실적이고 예술적인 이미지를 생성하는 방법을 배우게 됩니다. 이는 귀여운 강아지의 이미지처럼 간단할 수도 있고, 극적인 일몰 조명과 긴 그림자가 있는 시네마틱한 분위기의 greg rutkowski 스타일로 pixer가 아름답게 장식한 아늑한 주방의 초현실적 이미지처럼 상세할 수도 있습니다. 이는 이커머스 비즈니스 요구를 위한 제품 디자인 및 카탈로그 구축, 또는 사실적인 예술 작품이나 스톡 이미지 생성에 활용될 수 있습니다.


모델 라이선스: 이 모델을 사용함으로써, 귀하는 [CreativeML Open RAIL-M++ 라이선스](https://huggingface.co/stabilityai/stable-diffusion-2/blob/main/LICENSE-MODEL)에 동의하는 것입니다.

***

1. [설정](#1.-Set-Up)
2. [사전 훈련된 모델로 추론 실행](#2.-Run-inference-on-the-pre-trained-model)
    * [모델 선택](#2.1.-Select-a-Model)
    * [JumpStart 아티팩트 검색 및 엔드포인트 배포](#2.2.-Retrieve-JumpStart-Artifacts-&-Deploy-an-Endpoint)
    * [엔드포인트 쿼리 및 응답 파싱](#2.3.-Query-endpoint-and-parse-response)
    * [지원되는 추론 매개변수](#2.4.-Supported-Inference-parameters)
    * [압축된 이미지 출력](#2.5.-Compressed-Image-Output)
    * [프롬프트 엔지니어링](#2.6.-Prompt-Engineering)

참고: 이 노트북은 Python 3(Data Science 2.0) 커널이 있는 Amazon SageMaker Studio의 ml.t3.medium 인스턴스와 conda_python3 커널이 있는 Amazon SageMaker Notebook 인스턴스에서 테스트되었습니다. Workshop Studio 계정을 사용하는 경우 Data Science 2.0 커널을 사용하고 있는지 확인하세요.

참고: 사전 훈련된 모델을 배포하려면 `ml.g4dn.2xlarge` 인스턴스 유형을 사용할 수 있습니다. 해당 리전에서 `ml.g5.2xlarge`를 사용할 수 있다면, 배포에 해당 인스턴스 유형을 사용하는 것을 권장합니다.

### 1. Stable diffusion 모델 배포하기

***
노트북을 실행하기 전에 설정을 위한 몇 가지 초기 단계가 필요합니다. 이 노트북은 sagemaker와 ipywidgets의 최신 버전이 필요합니다.

***

In [ ]:
!pip install ipywidgets==7.0.0 --quiet
!pip install --upgrade pip
!pip install --upgrade sagemaker
!pip install langchain
!pip install datasets

#### 권한 및 환경 변수

***
Amazon SageMaker에서 호스팅하기 위해서는 AWS 서비스 사용을 설정하고 인증해야 합니다. 여기서는 현재 노트북과 연결된 실행 역할을 SageMaker 접근 권한이 있는 AWS 계정 역할로 사용합니다.

***

In [ ]:
from sagemaker.predictor import Predictor
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

In [ ]:
model_id = 'model-imagegeneration-stabilityai-stable-diffusion-xl-base-1-0'

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

try:
    model = JumpStartModel(model_id=model_id, instance_type="ml.g5.4xlarge")
    predictor = model.deploy()
except Exception as e:
    print(str(e))

In [ ]:
endpoint_name =predictor.endpoint_name
region = aws_region
print(f"SageMaker Endpoint with Stable Diffusion deployed: {endpoint_name}")

## 2. 사전 훈련된 모델로 추론 실행하기

***

JumpStart를 사용하면 새로운 데이터셋으로 먼저 미세 조정하지 않고도 사전 훈련된 모델에서 추론을 수행할 수 있습니다.
***

In [ ]:
# Create a SageMaker predictor object
model_predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

model_predictor.endpoint 

### 2.2 Stable Diffusion 엔드포인트 쿼리하기

### 지원되는 기능

***
이 모델은 추론을 수행하는 동안 많은 고급 매개변수를 지원합니다. 다음과 같은 매개변수가 포함됩니다:

* **text**: 이미지 생성을 안내하는 프롬프트. 반드시 지정해야 하며 문자열이어야 합니다.
* **width**: 생성된 이미지의 너비. 지정된 경우 8로 나눌 수 있는 양의 정수여야 합니다.
* **height**: 생성된 이미지의 높이. 지정된 경우 8로 나눌 수 있는 양의 정수여야 합니다. 이미지 크기는 256x256보다 커야 합니다.
* **sampler**: 사용 가능한 샘플러는 EulerEDMSampler, HeunEDMSampler, EulerAncestralSampler, DPMPP2SAncestralSampler, DPMPP2MSampler, LinearMultistepSampler입니다.
* **cfg_scale**: cfg_scale 값이 높을수록 이미지 품질을 희생하면서 프롬프트와 더 밀접하게 관련된 이미지가 생성됩니다. 지정된 경우 부동 소수점이어야 합니다. cfg_scale<=1은 무시됩니다.
* **steps**: 이미지 생성 중 디노이징 단계 수. 단계가 많을수록 이미지 품질이 높아집니다. 지정된 경우 양의 정수여야 합니다.
* **seed**: 재현성을 위해 무작위 상태를 고정합니다. 지정된 경우 정수여야 합니다.
* **use_refiner**: 리파이너는 기본적으로 SDXL 모델과 함께 사용됩니다. 이 매개변수를 사용하여 비활성화할 수 있습니다.
* **init_image**: 시작점으로 사용할 이미지.
* **image_strength**: 참조 이미지를 변환하는 정도를 나타냅니다. 0과 1 사이여야 합니다.
* **refiner_steps**: 리파이너의 이미지 생성 중 디노이징 단계 수. 단계가 많을수록 이미지 품질이 높아집니다. 지정된 경우 양의 정수여야 합니다.
* **refiner_strength**: 리파이너에 입력 이미지를 변환하는 정도를 나타냅니다.
* **negative_prompt**: 이 프롬프트에 반하는 이미지 생성을 안내합니다. 지정된 경우 문자열이어야 합니다. 음수 가중치와 함께 text_prompts에 지정됩니다.


***
### 텍스트에서 이미지로

***

In [ ]:
from PIL import Image
import io
import base64
import json
import boto3
from typing import Union, Tuple
import os

In [ ]:
payload = {
    "text_prompts": [{"text": "jaguar in the Amazon rainforest"}],
    "width": 1024,
    "height": 1024,
    "sampler": "DPMPP2MSampler",
    "cfg_scale": 7.0,
    "steps": 50,
    "seed": 133,
    "use_refiner": True,
    "refiner_steps": 40,
    "refiner_strength": 0.2,
}

In [ ]:
def decode_and_show(model_response) -> None:
    """
    Decodes and displays an image from SDXL output

    Args:
        model_response (GenerationResponse): The response object from the deployed  model.

    Returns:
        None
    """
    image = Image.open(io.BytesIO(base64.b64decode(model_response)))
    display(image)
    image.close()


response = model_predictor.predict(payload)
# If you get time out error please check the endpoint logs in cloudWatch for model loading status
# and invoke it again
decode_and_show(response["generated_image"])

### 이미지에서 이미지로

***
이미지를 입력으로 사용하는 추론을 수행하려면 이미지를 base64로 인코딩된 문자열로 init_image에 전달해야 합니다.

다음은 이미지를 base64로 인코딩된 문자열로 변환하는 도우미 함수입니다:

***### 2.4. 지원되는 추론 매개변수

***
이 모델은 추론을 수행하는 동안 많은 고급 매개변수도 지원합니다. 다음과 같은 매개변수가 있습니다:

* **prompt**: 이미지 생성을 안내하는 프롬프트. 반드시 지정해야 하며 문자열 또는 문자열 목록일 수 있습니다.
* **width**: 생성된 이미지의 너비. 지정된 경우 8로 나눌 수 있는 양의 정수여야 합니다.
* **height**: 생성된 이미지의 높이. 지정된 경우 8로 나눌 수 있는 양의 정수여야 합니다.
* **num_inference_steps**: 이미지 생성 중 디노이징 단계 수. 단계가 많을수록 이미지 품질이 높아집니다. 지정된 경우 양의 정수여야 합니다.
* **guidance_scale**: 높은 가이던스 스케일은 이미지 품질을 희생하면서 프롬프트와 밀접하게 관련된 이미지를 생성합니다. 지정된 경우 부동 소수점이어야 합니다. guidance_scale<=1은 무시됩니다.
* **negative_prompt**: 이 프롬프트에 반하는 이미지 생성을 안내합니다. 지정된 경우 문자열 또는 문자열 목록이어야 하며 guidance_scale과 함께 사용됩니다. guidance_scale이 비활성화된 경우 이것도 비활성화됩니다. 또한 prompt가 문자열 목록인 경우 negative_prompt도 문자열 목록이어야 합니다.
* **num_images_per_prompt**: 프롬프트당 반환되는 이미지 수. 지정된 경우 양의 정수여야 합니다.
* **seed**: 재현성을 위해 무작위 상태를 고정합니다. 지정된 경우 정수여야 합니다.

***

In [ ]:
def encode_image(
    image_path: str, resize: bool = True, size: Tuple[int, int] = (1024, 1024)
) -> Union[str, None]:
    """
    Encode an image as a base64 string, optionally resizing it to a supported resolution.

    Args:
        image_path (str): The path to the image file.
        resize (bool, optional): Whether to resize the image. Defaults to True.

    Returns:
        Union[str, None]: The encoded image as a string, or None if encoding failed.
    """
    assert os.path.exists(image_path)

    if resize:
        image = Image.open(image_path)
        image = image.resize(size)
        image.save("image_path_resized.png")
        image_path = "image_path_resized.png"
    image = Image.open(image_path)
    assert image.size == size
    with open(image_path, "rb") as image_file:
        img_byte_array = image_file.read()
        # Encode the byte array as a Base64 string
        try:
            base64_str = base64.b64encode(img_byte_array).decode("utf-8")
            return base64_str
        except Exception as e:
            print(f"Failed to encode image {image_path} as base64 string.")
            print(e)
            return None
    image.close()

이번에는 이미지와 프롬프트를 모두 모델에 입력해 보겠습니다. image_strength를 설정하여 이미지와 프롬프트의 상대적 중요도를 조정할 수 있습니다. 데모를 위해 강아지 사진을 사용하겠습니다.

In [ ]:
# Here is the original image:
region = boto3.Session().region_name
s3_bucket = f"jumpstart-cache-prod-{region}"
key_prefix = "model-metadata/assets"
input_img_file_name = "dog_suit.jpg"

s3 = boto3.client("s3")

s3.download_file(s3_bucket, f"{key_prefix}/{input_img_file_name}", input_img_file_name)
image = Image.open(input_img_file_name)
display(image)
image.close()

In [ ]:
size = (512, 512)
dog_data = encode_image(input_img_file_name, size=size)

payload = {
    "text_prompts": [{"text": "dog in embroidery"}],
    "init_image": dog_data,
    "cfg_scale": 9,
    "image_strength": 0.8,
    "seed": 42,
}

response = model_predictor.predict(payload)
decode_and_show(response["generated_image"])

### 2.6. 프롬프트 엔지니어링
---
좋은 프롬프트를 작성하는 것은 때로 예술과 같습니다. 특정 프롬프트가 주어진 모델에서 만족스러운 이미지를 생성할지 예측하기는 종종 어렵습니다. 그러나 효과적인 것으로 관찰된 특정 템플릿들이 있습니다. 일반적으로 프롬프트는 크게 세 부분으로 나눌 수 있습니다: (i) 이미지 유형(사진/스케치/그림 등), (ii) 설명(주제/객체/환경/장면 등) 및 (iii) 이미지 스타일(사실적/예술적/예술 유형 등). 이 세 부분을 개별적으로 변경하여 이미지의 변형을 생성할 수 있습니다. 형용사는 이미지 생성 과정에서 중요한 역할을 하는 것으로 알려져 있습니다. 또한, 더 많은 세부 사항을 추가하면 생성 과정에 도움이 됩니다.

사실적인 이미지를 생성하려면 "a photo of", "a photograph of", "realistic" 또는 "hyper realistic"과 같은 문구를 사용할 수 있습니다. 예술가의 작품 스타일로 이미지를 생성하려면 "by Pablo Piccaso" 또는 "oil painting by Rembrandt" 또는 "landscape art by Frederic Edwin Church" 또는 "pencil drawing by Albrecht Dürer"와 같은 문구를 사용할 수 있습니다. 다양한 예술가의 스타일을 조합할 수도 있습니다. 카테고리별 예술적 이미지를 생성하려면 "lion on a beach, abstract"와 같이 프롬프트에 예술 카테고리를 추가할 수 있습니다. 다른 카테고리로는 "oil painting", "pencil drawing", "pop art", "digital art", "anime", "cartoon", "futurism", "watercolor", "manga" 등이 있습니다. 또한 35mm 와이드 렌즈나 85mm 와이드 렌즈와 같은 조명이나 카메라 렌즈에 대한 세부 정보와 프레이밍(인물/풍경/클로즈업 등)에 대한 세부 정보를 포함할 수 있습니다.

모델은 동일한 프롬프트가 여러 번 주어져도 다른 이미지를 생성한다는 점에 유의하세요. 따라서 여러 이미지를 생성하고 응용 프로그램에 가장 적합한 이미지를 선택할 수 있습니다.

---

In [ ]:
prompts = [
    "a beautiful illustration of a young cybertronic hyderabadi american woman, round face, cateye glasses, purple colors, intricate, sharp focus, illustration, highly detailed, digital painting, concept art, matte, art by wlop and artgerm and greg rutkowski and alphonse mucha, masterpiece",
    "a photorealistic hyperrealistic render of an interior of a beautifully decorated cozy kitchen by pixar, greg rutkowski, wlop, artgerm, dramatic moody sunset lighting, long shadows, volumetric, cinematic atmosphere, octane render, artstation, 8 k",
    "symmetry!! portrait of nicolas cage, long hair in the wind, smile, happy, white vest, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha",
    "a stunningly detailed stained glass window of a beautiful poison ivy with green skin wearing a business suit, dark eyeliner, intricate, elegant, highly detailed, digital painting, artstation, concept art, sharp focus, illustration, art by greg rutkowski and alphonse mucha",
    "a fantasy style portrait painting of rachel lane / alison brie / sally kellerman hybrid in the style of francois boucher oil painting unreal 5 daz. rpg portrait, extremely detailed artgerm greg rutkowski alphonse mucha",
    "symmetry!! portrait of vanessa hudgens in the style of horizon zero dawn, machine face, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha, 8 k",
    "landscape of the beautiful city of paris rebuilt near the pacific ocean in sunny california, amazing weather, sandy beach, palm trees, splendid haussmann architecture, digital painting, highly detailed, intricate, without duplication, art by craig mullins, greg rutkwowski, concept art, matte painting, trending on artstation",
]
for prompt in prompts:
    payload = {"text_prompts": [{"text": prompt}],
    "width": 1024,
    "height": 1024,
    "sampler": "DPMPP2MSampler",
    "cfg_scale": 7.0,
    "steps": 50,
    "seed": 133,
    "use_refiner": True,
    "refiner_steps": 40,
    "refiner_strength": 0.2,}
    response = model_predictor.predict(payload)
    decode_and_show(response["generated_image"])

In [ ]:
# Delete the SageMaker endpoint
model_predictor.delete_model()
model_predictor.delete_endpoint()